# Westminster Ground Truth Analysis with MetaShape

This notebook demonstrates the complete workflow for creating orthomosaics from DJI drone imagery using **Agisoft MetaShape** and evaluating their accuracy:

1. **Data Loading**: Load images and GCPs from CSV
2. **GCP Conversion**: Convert UTM coordinates to WGS84 lat/lon for MetaShape
3. **Orthomosaic Creation**: Generate orthomosaics with and without GCPs using MetaShape
4. **Basemap Comparison**: Download basemaps and quantify absolute accuracy
5. **Quality Report**: Generate comprehensive comparison report

## Datasets:
- **Dataset 1**: DJI_202510060955_017_25-3288 (543 images)
- **Dataset 2**: DJI_202510060955_019_25-3288 (528 images)

For each dataset, we create:
- Orthomosaic **without** GCPs
- Orthomosaic **with** GCPs

All orthomosaics are compared against reference basemaps to evaluate accuracy.


## Setup: Install Dependencies

First, install the required packages. Note: This notebook requires **Agisoft MetaShape Python API** to be installed separately.


In [1]:
# Install required packages
import subprocess
import sys
from pathlib import Path

# Try to install from requirements.txt first
requirements_file = Path("requirements.txt")
if requirements_file.exists():
    print("Installing packages from requirements.txt...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_file)])
    print("✓ Packages installed from requirements.txt")
else:
    # Fallback: install packages individually
    print("requirements.txt not found. Installing packages individually...")
    packages = [
        "numpy>=1.24.0",
        "rasterio>=1.3.0",
        "pillow>=10.0.0",
        "matplotlib>=3.7.0",
        "pandas>=2.0.0",
        "pyproj>=3.6.0",
        "requests>=2.31.0",
        "utm>=0.7.0",
    ]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("✓ All packages installed")

# Check for MetaShape
try:
    import Metashape
    print("✓ MetaShape Python API is available")
    METASHAPE_AVAILABLE = True
except ImportError:
    print("⚠️  MetaShape Python API not found. Please install Agisoft MetaShape and its Python API.")
    print("   The API is typically installed with MetaShape at:")
    print("   - Windows: C:\\Program Files\\Agisoft\\Metashape Pro\\python")
    print("   - macOS: /Applications/Metashape Pro/Metashape.app/Contents/Frameworks/Python.framework/Versions/3.9")
    print("   - Linux: /opt/metashape-pro/lib/python3.9")
    METASHAPE_AVAILABLE = False

print("\nSetup complete!")


Installing packages from requirements.txt...
✓ Packages installed from requirements.txt
✓ MetaShape Python API is available

Setup complete!


## Imports


In [2]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
import logging
import json
import csv
import xml.etree.ElementTree as ET
import utm
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Add package to path
sys.path.insert(0, str(Path.cwd()))

# Import Westminster-specific modules
from westminster_ground_truth_analysis import (
    GCPParser,
    download_basemap,
    compare_orthomosaic_to_basemap,
)

# Try to import MetaShape processor from qualicum_beach package
# If not available, we'll define the functions locally
try:
    from qualicum_beach_gcp_analysis import (
        process_orthomosaic,
        PhotoMatchQuality,
        DepthMapQuality,
        export_to_metashape_csv,
        export_to_metashape_xml,
    )
    print("✓ Using MetaShape processor from qualicum_beach_gcp_analysis")
    USE_QUALICUM_PACKAGE = True
except ImportError:
    print("⚠️  qualicum_beach_gcp_analysis not found. Will define MetaShape functions locally.")
    USE_QUALICUM_PACKAGE = False
    # We'll define these functions in the next cell if needed

# Set up paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

print("✓ Imports successful!")


⚠️  qualicum_beach_gcp_analysis not found. Will define MetaShape functions locally.
✓ Imports successful!


## Step 1: Load Ground Control Points


In [3]:
# Parse GCP file (UTM coordinates)
gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
gcp_parser = GCPParser(str(gcp_file))

gcps_utm = gcp_parser.get_gcps()
print(f"Loaded {len(gcps_utm)} ground control points (UTM format)")

# Display first few GCPs
print("\nFirst few GCPs (UTM):")
for gcp in gcps_utm[:5]:
    print(f"  {gcp.name}: X={gcp.x:.2f}, Y={gcp.y:.2f}, Z={gcp.z:.2f}")

# Get bounds
min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
print(f"\nGCP Bounds (UTM): X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")


Loaded 23 ground control points (UTM format)

First few GCPs (UTM):
  GCP1: X=5450945.53, Y=506914.12, Z=77.45
  GCP2: X=5450730.01, Y=506657.79, Z=79.22
  GCP3: X=5450480.01, Y=506577.77, Z=59.40
  GCP4: X=5450578.63, Y=506765.03, Z=65.59
  GCP5: X=5450715.96, Y=506926.13, Z=63.10

GCP Bounds (UTM): X=[5450109.82, 5450992.66], Y=[506577.77, 507315.01]


## Step 2: Convert GCPs to WGS84 Lat/Lon for MetaShape

MetaShape expects GCPs in WGS84 lat/lon format. We need to convert from UTM Zone 10N.


In [4]:
# Convert UTM to WGS84 lat/lon
# NOTE: In the CSV, X column is actually Northing (~5.45M) and Y column is Easting (~500k)
# utm.to_latlon expects (easting, northing), so we need to swap them
gcps_wgs84 = []

for gcp in gcps_utm:
    # Convert UTM to lat/lon (UTM Zone 10N)
    # X is Northing, Y is Easting
    lat, lon = utm.to_latlon(gcp.y, gcp.x, 10, 'N')
    
    gcp_dict = {
        'id': gcp.name,
        'label': gcp.name,
        'lat': lat,
        'lon': lon,
        'z': gcp.z,
        'accuracy': 0.1  # Default accuracy in meters
    }
    gcps_wgs84.append(gcp_dict)

print(f"Converted {len(gcps_wgs84)} GCPs to WGS84 lat/lon")
print("\nFirst few GCPs (WGS84):")
for gcp in gcps_wgs84[:5]:
    print(f"  {gcp['id']}: ({gcp['lat']:.6f}, {gcp['lon']:.6f}, z={gcp['z']:.2f})")


Converted 23 GCPs to WGS84 lat/lon

First few GCPs (WGS84):
  GCP1: (49.211262, -122.905068, z=77.45)
  GCP2: (49.209326, -122.908591, z=79.22)
  GCP3: (49.207078, -122.909694, z=59.40)
  GCP4: (49.207963, -122.907122, z=65.59)
  GCP5: (49.209197, -122.904907, z=63.10)


## Step 3: Export GCPs for MetaShape


In [ ]:
# Create output directory for GCP files
gcp_output_dir = output_dir / "gcps"
gcp_output_dir.mkdir(exist_ok=True)

# Export to MetaShape XML format (preferred by MetaShape)
if USE_QUALICUM_PACKAGE:
    gcp_xml_path = gcp_output_dir / "gcps_metashape.xml"
    export_to_metashape_xml(gcps_wgs84, str(gcp_xml_path))
    print(f"✓ GCPs exported to XML: {gcp_xml_path}")
    
    # Also export CSV for reference
    gcp_csv_path = gcp_output_dir / "gcps_metashape.csv"
    export_to_metashape_csv(gcps_wgs84, str(gcp_csv_path))
    print(f"✓ GCPs also exported to CSV: {gcp_csv_path}")
else:
    # Define export functions locally if qualicum package not available
    def export_to_metashape_xml_local(gcps, output_path):
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        root = ET.Element('document')
        chunks = ET.SubElement(root, 'chunks')
        chunk = ET.SubElement(chunks, 'chunk')
        markers = ET.SubElement(chunk, 'markers')
        
        for gcp in gcps:
            marker = ET.SubElement(markers, 'marker')
            marker.set('label', gcp.get('id', gcp.get('label', 'GCP')))
            marker.set('reference', 'true')
            
            position = ET.SubElement(marker, 'position')
            position.set('x', str(gcp.get('lon', 0.0)))
            position.set('y', str(gcp.get('lat', 0.0)))
            position.set('z', str(gcp.get('z', 0.0)))
            
            accuracy = gcp.get('accuracy', 1.0)
            accuracy_elem = ET.SubElement(marker, 'accuracy')
            accuracy_elem.set('x', str(accuracy))
            accuracy_elem.set('y', str(accuracy))
            accuracy_elem.set('z', str(accuracy))
        
        tree = ET.ElementTree(root)
        ET.indent(tree, space='  ')
        tree.write(output_path, encoding='utf-8', xml_declaration=True)
        print(f"Exported {len(gcps)} GCPs to MetaShape XML: {output_path}")
        return str(output_path)
    
    def export_to_metashape_csv_local(gcps, output_path):
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(['Label', 'X', 'Y', 'Z', 'Accuracy', 'Enabled'])
            
            for gcp in gcps:
                label = gcp.get('id', gcp.get('label', 'GCP'))
                lon = gcp.get('lon', 0.0)
                lat = gcp.get('lat', 0.0)
                z = gcp.get('z', 0.0)
                accuracy = gcp.get('accuracy', 1.0)
                writer.writerow([label, lon, lat, z, accuracy, '1'])
        
        print(f"Exported {len(gcps)} GCPs to MetaShape CSV: {output_path}")
        return str(output_path)
    
    gcp_xml_path = gcp_output_dir / "gcps_metashape.xml"
    export_to_metashape_xml_local(gcps_wgs84, str(gcp_xml_path))
    print(f"✓ GCPs exported to XML: {gcp_xml_path}")
    
    gcp_csv_path = gcp_output_dir / "gcps_metashape.csv"
    export_to_metashape_csv_local(gcps_wgs84, str(gcp_csv_path))
    print(f"✓ GCPs also exported to CSV: {gcp_csv_path}")

# Use CSV file for processing (more reliable than XML)
# If CSV doesn't exist, fall back to XML
if gcp_csv_path.exists():
    gcp_file_for_processing = gcp_csv_path
    print(f"✓ Using CSV format for GCP import: {gcp_csv_path}")
else:
    gcp_file_for_processing = gcp_xml_path
    print(f"✓ Using XML format for GCP import: {gcp_xml_path}")


Exported 23 GCPs to MetaShape XML: outputs/gcps/gcps_metashape.xml
✓ GCPs exported to XML: outputs/gcps/gcps_metashape.xml
Exported 23 GCPs to MetaShape CSV: outputs/gcps/gcps_metashape.csv
✓ GCPs also exported to CSV: outputs/gcps/gcps_metashape.csv


In [ ]:
# Define MetaShape processing functions if qualicum package not available
# USE_STANDARD_PIPELINE_PARAMETERS indicates if we're using the qualicum package's standard pipeline
# If not available, we define the functions locally with the same parameters
try:
    USE_STANDARD_PIPELINE_PARAMETERS = USE_QUALICUM_PACKAGE
except NameError:
    USE_STANDARD_PIPELINE_PARAMETERS = False

# Check if MetaShape is available (defined in setup cell)
try:
    _ = METASHAPE_AVAILABLE
except NameError:
    METASHAPE_AVAILABLE = False

if not USE_STANDARD_PIPELINE_PARAMETERS and METASHAPE_AVAILABLE:
    from enum import IntEnum
    from typing import Optional
    from contextlib import contextmanager
    
    class PhotoMatchQuality(IntEnum):
        LowestQuality = 0
        LowQuality = 1
        MediumQuality = 2
        HighQuality = 4
        HighestQuality = 8
    
    class DepthMapQuality(IntEnum):
        LowestQuality = 1
        LowQuality = 2
        MediumQuality = 4
        HighQuality = 8
        UltraQuality = 16
    
    @contextmanager
    def redirect_metashape_output(log_file_path: Path):
        """Context manager to redirect MetaShape's stdout/stderr to a log file.
        
        MetaShape prints verbose output to stdout/stderr. This function redirects
        that output to a log file while keeping minimal progress info in the notebook.
        """
        log_file_path.parent.mkdir(parents=True, exist_ok=True)
        # Open in write mode to start fresh each time (or append if you want to keep history)
        log_file = open(log_file_path, 'w', encoding='utf-8')
        original_stdout = sys.stdout
        original_stderr = sys.stderr
        
        # Create a Tee-like object that writes to both file and original stdout for progress
        class TeeOutput:
            def __init__(self, file, original):
                self.file = file
                self.original = original
                self.buffer = []
            
            def write(self, text):
                # Always write to log file
                self.file.write(text)
                self.file.flush()  # Flush immediately so log file is up to date
                
                # Only show minimal progress in notebook (filter verbose output)
                # MetaShape progress lines typically contain "Progress:" or percentages
                if any(keyword in text for keyword in ['Progress:', '%', '✓', '✗', '⚠️', '📝', '📂', '📷', '🔍', '📐', '🗺️', '🏗️', '🖼️', '💾']):
                    # Show progress indicators in notebook
                    self.original.write(text)
                    self.original.flush()
                # Suppress verbose technical output in notebook
            
            def flush(self):
                self.file.flush()
                self.original.flush()
        
        tee_stdout = TeeOutput(log_file, original_stdout)
        tee_stderr = TeeOutput(log_file, original_stderr)
        
        try:
            sys.stdout = tee_stdout
            sys.stderr = tee_stderr
            yield log_file
        finally:
            sys.stdout = original_stdout
            sys.stderr = original_stderr
            log_file.flush()
            log_file.close()
    
    def process_orthomosaic(
        photos_dir: Path,
        output_path: Path,
        project_path: Path,
        gcp_file: Optional[Path] = None,
        product_id: str = "orthomosaic",
        clean_intermediate_files: bool = False,
        photo_match_quality: int = PhotoMatchQuality.MediumQuality,
        depth_map_quality: int = DepthMapQuality.MediumQuality,
        tiepoint_limit: int = 10000,
        use_gcps: bool = False
    ) -> dict:
        """Process orthomosaic using MetaShape with output redirected to log file."""
        import Metashape
        
        # Configure GPU
        Metashape.app.gpu_mask = ~0
        
        # Setup paths
        output_path.mkdir(parents=True, exist_ok=True)
        project_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Setup log file for MetaShape verbose output
        log_dir = project_path.parent / "logs"
        log_dir.mkdir(parents=True, exist_ok=True)
        log_file_path = log_dir / f"{product_id}_metashape.log"
        
        print(f"📝 MetaShape verbose output will be saved to: {log_file_path}")
        
        # Check if project exists
        project_exists = project_path.exists()
        
        # Helper function to safely save document
        def safe_save_document():
            """Save document, handling read-only errors by recreating the document."""
            nonlocal doc, chunk
            try:
                doc.save(str(project_path))
            except OSError as e:
                if "read-only" in str(e).lower() or "editing is disabled" in str(e).lower():
                    print(f"  ⚠️  Document became read-only, recreating...")
                    # Close the read-only document
                    try:
                        doc.close()
                    except:
                        pass
                    # Create a new writable document and copy the chunk data
                    # Note: We can't easily copy chunk data, so we'll just create a new document
                    # This means we lose the current processing state, but at least we can continue
                    print(f"  ⚠️  Warning: Processing state may be lost. Consider using clean_intermediate_files=True next time.")
                    doc = Metashape.Document()
                    doc.save(str(project_path))
                    # Get the chunk from the new document
                    if len(doc.chunks) > 0:
                        chunk = doc.chunks[0]
                    else:
                        chunk = doc.addChunk()
                else:
                    raise
        
        # Use context manager to redirect MetaShape output to log file
        with redirect_metashape_output(log_file_path):
            # Close any existing document first to avoid "already in use" error
            try:
                # Close all open documents
                if hasattr(Metashape.app, 'document') and Metashape.app.document is not None:
                    Metashape.app.document.close()
                # Also try to close via app.documents if available
                if hasattr(Metashape.app, 'documents'):
                    for d in list(Metashape.app.documents):
                        try:
                            d.close()
                        except:
                            pass
            except:
                pass  # Ignore errors when closing
            
            doc = None
            if project_exists and not clean_intermediate_files:
                print(f"📂 Loading existing project: {project_path}")
                doc = Metashape.Document()
                try:
                    doc.open(str(project_path))
                    # Check if document is read-only - if so, we need to close and recreate
                    # Try a test save to see if it's writable
                    try:
                        doc.save(str(project_path))
                        doc_readonly = False
                    except OSError as e:
                        if "read-only" in str(e).lower() or "editing is disabled" in str(e).lower():
                            print(f"  ⚠️  Document is read-only, closing and creating new project...")
                            doc.close()
                            doc = None
                            doc_readonly = True
                        else:
                            raise
                    
                    if doc is not None and not doc_readonly:
                        if len(doc.chunks) > 0:
                            chunk = doc.chunks[0]
                            print(f"  ✓ Found existing chunk with {len(chunk.cameras)} cameras")
                        else:
                            chunk = doc.addChunk()
                            print("  ✓ Created new chunk")
                except Exception as e:
                    # If open fails (e.g., file locked), try to create new project
                    print(f"  ⚠️  Could not open existing project: {e}")
                    print("  Creating new project instead...")
                    if doc is not None:
                        try:
                            doc.close()
                        except:
                            pass
                    doc = None
            
            # Create new document if we don't have one yet
            if doc is None:
                if clean_intermediate_files and project_exists:
                    try:
                        project_path.unlink()
                    except:
                        pass  # Ignore if can't delete
                print("🚀 Creating new MetaShape project...")
                doc = Metashape.Document()
                doc.save(str(project_path))
                chunk = doc.addChunk()
            
            # Add photos
            if len(chunk.cameras) == 0:
                print(f"📷 Adding photos from: {photos_dir}")
                photos = list(photos_dir.glob("*.jpg")) + list(photos_dir.glob("*.JPG"))
                if not photos:
                    raise ValueError(f"No images found in {photos_dir}")
                chunk.addPhotos([str(p) for p in photos])
                safe_save_document()  # Save after adding photos
                print(f"  ✓ Added {len(photos)} photos")
            else:
                print(f"  ✓ Photos already added ({len(chunk.cameras)} cameras)")
            
            # Add GCPs if requested
            if use_gcps and gcp_file and gcp_file.exists():
                if len(chunk.markers) == 0:
                    print(f"📍 Loading GCPs from: {gcp_file}")
                    
                    # Check file extension to determine format
                    file_ext = Path(gcp_file).suffix.lower()
                    markers_added = 0
                    
                    if file_ext == '.xml':
                        # Try XML format first
                        try:
                            chunk.importMarkers(str(gcp_file))
                            markers_added = len(chunk.markers)
                            print(f"  ✓ Added {markers_added} markers from XML")
                        except Exception as e:
                            print(f"  ⚠️  XML import failed: {e}")
                            print(f"  Trying CSV format instead...")
                            # Fall back to CSV if XML fails
                            file_ext = '.csv'
                    
                    if file_ext == '.csv' or file_ext == '.txt':
                        # CSV format - read and add markers manually
                        import csv
                        with open(gcp_file, 'r', encoding='utf-8') as f:
                            reader = csv.DictReader(f, delimiter='\t')
                            for row in reader:
                                try:
                                    label = row.get('Label', row.get('label', ''))
                                    x = float(row.get('X', row.get('x', 0)))
                                    y = float(row.get('Y', row.get('y', 0)))
                                    z = float(row.get('Z', row.get('z', 0)))
                                    accuracy = float(row.get('Accuracy', row.get('accuracy', 1.0)))
                                    enabled = row.get('Enabled', row.get('enabled', '1')).strip() == '1'
                                    
                                    # Create marker
                                    marker = chunk.addMarker()
                                    marker.label = label
                                    marker.reference.location = Metashape.Vector([x, y, z])
                                    marker.reference.accuracy = Metashape.Vector([accuracy, accuracy, accuracy])
                                    marker.reference.enabled = enabled
                                    markers_added += 1
                                except Exception as e:
                                    print(f"  ⚠️  Error adding marker from row: {e}")
                                    continue
                        
                        if markers_added > 0:
                            print(f"  ✓ Added {markers_added} markers from CSV")
                        else:
                            raise ValueError(f"Failed to add any markers from {gcp_file}")
                    
                    safe_save_document()  # Save after adding GCPs
                    print(f"  ✓ Total markers: {len(chunk.markers)}")
                else:
                    print(f"  ✓ GCPs already loaded ({len(chunk.markers)} markers)")
            
            # Match photos
            # Check if tie points exist - MetaShape TiePoints object may not support len()
            # Also check if cameras are already aligned (which requires tie points)
            tie_points_exist = chunk.tie_points is not None
            cameras_aligned = sum(1 for cam in chunk.cameras if cam.transform) > 0
            
            # If cameras are aligned, we definitely have tie points (even if we can't count them)
            if cameras_aligned:
                tie_points_exist = True
                tie_points_count = 1  # Don't need exact count, just know they exist
            elif tie_points_exist:
                # Try to get count, but handle case where len() doesn't work on TiePoints object
                try:
                    tie_points_count = len(chunk.tie_points)
                except (TypeError, AttributeError):
                    # If len() doesn't work, check if we can iterate or get a count attribute
                    try:
                        # Try to get point count if available
                        tie_points_count = chunk.tie_points.point_count if hasattr(chunk.tie_points, 'point_count') else None
                        if tie_points_count is None:
                            # Try to check if it's iterable and count items
                            try:
                                tie_points_count = sum(1 for _ in chunk.tie_points)
                            except:
                                tie_points_count = 1  # Assume it has points if it exists
                    except:
                        tie_points_count = 1  # Assume it has points if it exists
            else:
                tie_points_count = 0
            
            if not tie_points_exist or tie_points_count == 0:
                print("🔍 Matching photos... (this may take a while)")
                chunk.matchPhotos(
                    downscale=photo_match_quality,
                    tiepoint_limit=tiepoint_limit,
                )
                safe_save_document()  # Save after matching photos
                # Try to get count after matching
                if chunk.tie_points is not None:
                    try:
                        tie_points_count = len(chunk.tie_points)
                    except (TypeError, AttributeError):
                        try:
                            tie_points_count = chunk.tie_points.point_count if hasattr(chunk.tie_points, 'point_count') else 0
                        except:
                            try:
                                tie_points_count = sum(1 for _ in chunk.tie_points)
                            except:
                                tie_points_count = 0
                else:
                    tie_points_count = 0
                print(f"  ✓ Photo matching complete ({tie_points_count} tie points)")
            else:
                print(f"  ✓ Photos already matched ({tie_points_count} tie points)")
            
            # Align cameras
            aligned = sum(1 for cam in chunk.cameras if cam.transform)
            cameras_aligned = aligned > 0
            if aligned == 0:
                print("📐 Aligning cameras... (this may take a while)")
                chunk.alignCameras()
                safe_save_document()
                aligned = sum(1 for cam in chunk.cameras if cam.transform)
                cameras_aligned = aligned > 0
                print(f"  ✓ Camera alignment complete ({aligned}/{len(chunk.cameras)} cameras aligned)")
            else:
                print(f"  ✓ Cameras already aligned ({aligned}/{len(chunk.cameras)} cameras)")
            
            # Build depth maps
            # Check if depth maps exist - cameras must be aligned first
            # DepthMaps object doesn't support len(), so just check if it exists
            if not cameras_aligned:
                print("  ⚠️  Skipping depth maps - cameras not aligned yet")
                depth_maps_exist = False
            else:
                depth_maps_exist = chunk.depth_maps is not None
                if not depth_maps_exist:
                    print("🗺️  Building depth maps... (this may take a while)")
                    chunk.buildDepthMaps(
                        downscale=depth_map_quality,
                        filter_mode=Metashape.MildFiltering
                    )
                    safe_save_document()  # Save after building depth maps
                    depth_maps_exist = chunk.depth_maps is not None
                    if depth_maps_exist:
                        # Try to get count for display, but handle if len() doesn't work
                        try:
                            depth_maps_count = len(chunk.depth_maps)
                        except (TypeError, AttributeError):
                            # Try alternative methods to get count
                            try:
                                depth_maps_count = chunk.depth_maps.count if hasattr(chunk.depth_maps, 'count') else None
                            except:
                                depth_maps_count = None
                            if depth_maps_count is None:
                                try:
                                    depth_maps_count = sum(1 for _ in chunk.depth_maps)
                                except:
                                    depth_maps_count = "?"
                        print(f"  ✓ Depth maps built ({depth_maps_count} depth maps)" if depth_maps_count != "?" else "  ✓ Depth maps built")
                    else:
                        print("  ⚠️  Depth map building may have failed")
                else:
                    # Try to get count for display
                    try:
                        depth_maps_count = len(chunk.depth_maps)
                    except (TypeError, AttributeError):
                        try:
                            depth_maps_count = chunk.depth_maps.count if hasattr(chunk.depth_maps, 'count') else "?"
                        except:
                            depth_maps_count = "?"
                        if depth_maps_count == "?":
                            try:
                                depth_maps_count = sum(1 for _ in chunk.depth_maps)
                            except:
                                depth_maps_count = "?"
                    print(f"  ✓ Depth maps already built ({depth_maps_count} depth maps)" if depth_maps_count != "?" else "  ✓ Depth maps already built")
            
            # Build model
            # Model requires depth maps
            if not depth_maps_exist:
                print("  ⚠️  Skipping model - depth maps not built yet")
            elif chunk.model is None:
                print("🏗️  Building 3D model... (this may take a while)")
                chunk.buildModel()
                safe_save_document()  # Save after building model
                print("  ✓ 3D model built")
            else:
                print("  ✓ 3D model already built")
            
            # Build orthomosaic
            # Orthomosaic requires model
            if chunk.model is None:
                print("  ⚠️  Skipping orthomosaic - model not built yet")
            elif chunk.orthomosaic is None:
                print("🖼️  Building orthomosaic... (this may take a while)")
                chunk.buildOrthomosaic()
                safe_save_document()  # Save after building orthomosaic
                print("  ✓ Orthomosaic built")
            else:
                print("  ✓ Orthomosaic already built")
            
            # Export GeoTIFF
            ortho_path = output_path / f"{product_id}.tif"
            if not ortho_path.exists() or clean_intermediate_files:
                print(f"💾 Exporting GeoTIFF to: {ortho_path}")
                compression = Metashape.ImageCompression()
                compression.tiff_compression = Metashape.ImageCompression.TiffCompressionNone
                compression.tiff_big = True
                chunk.exportRaster(str(ortho_path), image_compression=compression)
                safe_save_document()
                if ortho_path.exists():
                    file_size_mb = ortho_path.stat().st_size / (1024 * 1024)
                    print(f"  ✓ GeoTIFF exported ({file_size_mb:.2f} MB)")
                else:
                    print(f"  ⚠️  Export completed but file not found at: {ortho_path}")
            else:
                file_size_mb = ortho_path.stat().st_size / (1024 * 1024)
                print(f"  ✓ GeoTIFF already exists ({file_size_mb:.2f} MB)")
        
        stats = {
            'product_id': product_id,
            'use_gcps': use_gcps,
            'num_photos': len(chunk.cameras),
            'num_markers': len(chunk.markers),
            'ortho_path': str(ortho_path),
            'project_path': str(project_path),
            'log_file_path': str(log_file_path),
        }
        
        print(f"\n✅ Processing complete! Log file: {log_file_path}")
        return stats
    
    print("✓ MetaShape processing functions defined locally")
elif not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Cannot process orthomosaics.")
else:
    print("✓ Using MetaShape processor from qualicum_beach_gcp_analysis")


✓ MetaShape processing functions defined locally


In [7]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Setup paths for Dataset 1
    dataset1_dir = data_dir / "DJI_202510060955_017_25-3288"
    intermediate_dir = output_dir / "intermediate"
    ortho_output_dir = output_dir / "orthomosaics"
    
    # Process orthomosaic WITHOUT GCPs
    print("=" * 60)
    print("Processing Dataset 1 - WITHOUT GCPs...")
    print("=" * 60)
    
    project_path1_no_gcps = intermediate_dir / "dataset1_no_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats1_no_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_no_gcps,
            product_id="dataset1_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    else:
        stats1_no_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_no_gcps,
            product_id="dataset1_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    
    print("\n✓ Dataset 1 processing (without GCPs) complete!")
    print(f"  Number of photos: {stats1_no_gcps['num_photos']}")
    print(f"  Orthomosaic: {stats1_no_gcps['ortho_path']}")
    
    ortho1_no_gcps_path = Path(stats1_no_gcps['ortho_path'])


Processing Dataset 1 - WITHOUT GCPs...
📝 MetaShape verbose output will be saved to: outputs/intermediate/logs/dataset1_no_gcps_metashape.log
📂 Loading existing project: outputs/intermediate/dataset1_no_gcps.psxLoadProject: path = outputs/intermediate/dataset1_no_gcps.psx
loaded project in 0.000382 sec
SaveProject: path = outputs/intermediate/dataset1_no_gcps.psx
saved project in 0.071761 sec
LoadProject: path = outputs/intermediate/dataset1_no_gcps.psx
loaded project in 0.10347 sec


Document.open(): The document is opened in read-only mode because it is already in use.


  ✓ Found existing chunk with 543 cameras  ✓ Photos already added (543 cameras)  ✓ Photos already matched (1 tie points)  ✓ Cameras already aligned (543/543 cameras)  ✓ Depth maps already built🏗️  Building 3D model... (this may take a while)BuildModel: source data = Depth maps, surface type = Arbitrary, face count = High, volumetric masking = 0, OOC version, interpolation = Enabled, vertex colors = 1
Compression level: 1
Preparing depth maps...
543 depth maps
scheduled 28 depth map groups (543 cameras)
saved camera partition in 0.000532 sec
loaded camera partition in 0.000233 sec
saved group #1/28: done in 2.41528 s, 20 cameras, 145.586 MB data, 24.1406 KB registry
loaded camera partition in 8.4e-05 sec
saved group #2/28: done in 2.5573 s, 20 cameras, 170.501 MB data, 24.1406 KB registry
loaded camera partition in 8.4e-05 sec
saved group #3/28: done in 2.64373 s, 20 cameras, 184.71 MB data, 24.1406 KB registry
loaded camera partition in 8.3e-05 sec
saved group #4/28: done in 2.19159 s,

## Step 6: Process Dataset 1 - WITH GCPs


In [8]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Process orthomosaic WITH GCPs
    print("=" * 60)
    print("Processing Dataset 1 - WITH GCPs...")
    print("=" * 60)
    
    project_path1_with_gcps = intermediate_dir / "dataset1_with_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats1_with_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset1_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    else:
        stats1_with_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset1_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    
    print("\n✓ Dataset 1 processing (with GCPs) complete!")
    print(f"  Number of photos: {stats1_with_gcps['num_photos']}")
    print(f"  Number of markers: {stats1_with_gcps.get('num_markers', 0)}")
    print(f"  Orthomosaic: {stats1_with_gcps['ortho_path']}")
    
    ortho1_with_gcps_path = Path(stats1_with_gcps['ortho_path'])


Processing Dataset 1 - WITH GCPs...
📝 MetaShape verbose output will be saved to: outputs/intermediate/logs/dataset1_with_gcps_metashape.log
SaveProject: path = outputs/intermediate/dataset1_with_gcps.psx
saved project in 0.000453 sec
LoadProject: path = outputs/intermediate/dataset1_with_gcps.psx
loaded project in 0.000377 sec
📷 Adding photos from: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_017_25-3288AddPhotos
SaveProject: path = outputs/intermediate/dataset1_with_gcps.psx
saved project in 0.043499 sec
LoadProject: path = outputs/intermediate/dataset1_with_gcps.psx
loaded project in 0.031893 sec
  ✓ Added 543 photosImportMarkers: path = outputs/gcps/gcps_metashape.xml


RuntimeError: Invalid file format

## Step 7: Process Dataset 2 - WITHOUT GCPs


In [ ]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Setup paths for Dataset 2
    dataset2_dir = data_dir / "DJI_202510060955_019_25-3288"
    
    # Process orthomosaic WITHOUT GCPs
    print("=" * 60)
    print("Processing Dataset 2 - WITHOUT GCPs...")
    print("=" * 60)
    
    project_path2_no_gcps = intermediate_dir / "dataset2_no_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats2_no_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_no_gcps,
            product_id="dataset2_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    else:
        stats2_no_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_no_gcps,
            product_id="dataset2_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    
    print("\n✓ Dataset 2 processing (without GCPs) complete!")
    print(f"  Number of photos: {stats2_no_gcps['num_photos']}")
    print(f"  Orthomosaic: {stats2_no_gcps['ortho_path']}")
    
    ortho2_no_gcps_path = Path(stats2_no_gcps['ortho_path'])


## Step 8: Process Dataset 2 - WITH GCPs


In [ ]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Process orthomosaic WITH GCPs
    print("=" * 60)
    print("Processing Dataset 2 - WITH GCPs...")
    print("=" * 60)
    
    project_path2_with_gcps = intermediate_dir / "dataset2_with_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats2_with_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset2_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    else:
        stats2_with_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset2_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    
    print("\n✓ Dataset 2 processing (with GCPs) complete!")
    print(f"  Number of photos: {stats2_with_gcps['num_photos']}")
    print(f"  Number of markers: {stats2_with_gcps.get('num_markers', 0)}")
    print(f"  Orthomosaic: {stats2_with_gcps['ortho_path']}")
    
    ortho2_with_gcps_path = Path(stats2_with_gcps['ortho_path'])


## Step 9: Download Reference Basemap


In [ ]:
# Calculate bounding box from GCPs for basemap download
# Convert UTM bounds to lat/lon
center_easting = (min_y + max_y) / 2  # Y column is easting
center_northing = (min_x + max_x) / 2  # X column is northing
lat_center, lon_center = utm.to_latlon(center_easting, center_northing, 10, 'N')

# Convert bounds
lat_min, lon_min = utm.to_latlon(min_y, min_x, 10, 'N')
lat_max, lon_max = utm.to_latlon(max_y, max_x, 10, 'N')

# Add padding
padding = 0.001
bbox = (lat_min - padding, lon_min - padding, lat_max + padding, lon_max + padding)

print(f"Basemap bounding box: {bbox}")

# Download basemap
basemap_path = download_basemap(
    bbox=bbox,
    output_path=str(output_dir / "basemap.tif"),
    source="esri_world_imagery",
    target_resolution=0.1  # 0.1m per pixel
)

print(f"\n✓ Basemap saved to: {basemap_path}")


## Step 10: Compare Orthomosaics to Basemap


In [ ]:
# Compare all orthomosaics to basemap
comparison_dir = output_dir / "comparisons"
comparison_dir.mkdir(exist_ok=True)

# Determine orthomosaic paths
if 'ortho1_no_gcps_path' not in locals():
    ortho_output_dir = output_dir / "orthomosaics"
    ortho1_no_gcps_path = ortho_output_dir / "dataset1_no_gcps.tif"
    ortho1_with_gcps_path = ortho_output_dir / "dataset1_with_gcps.tif"
    ortho2_no_gcps_path = ortho_output_dir / "dataset2_no_gcps.tif"
    ortho2_with_gcps_path = ortho_output_dir / "dataset2_with_gcps.tif"

print("=" * 60)
print("Comparing orthomosaics to basemap...")
print("=" * 60)

# Compare Dataset 1 - Without GCPs
if ortho1_no_gcps_path.exists():
    print("\nDataset 1 - Without GCPs")
    print("-" * 60)
    metrics1_no_gcps = compare_orthomosaic_to_basemap(
        str(ortho1_no_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset1_no_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho1_no_gcps_path}")
    metrics1_no_gcps = {}

# Compare Dataset 1 - With GCPs
if ortho1_with_gcps_path.exists():
    print("\nDataset 1 - With GCPs")
    print("-" * 60)
    metrics1_with_gcps = compare_orthomosaic_to_basemap(
        str(ortho1_with_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset1_with_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho1_with_gcps_path}")
    metrics1_with_gcps = {}

# Compare Dataset 2 - Without GCPs
if ortho2_no_gcps_path.exists():
    print("\nDataset 2 - Without GCPs")
    print("-" * 60)
    metrics2_no_gcps = compare_orthomosaic_to_basemap(
        str(ortho2_no_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset2_no_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho2_no_gcps_path}")
    metrics2_no_gcps = {}

# Compare Dataset 2 - With GCPs
if ortho2_with_gcps_path.exists():
    print("\nDataset 2 - With GCPs")
    print("-" * 60)
    metrics2_with_gcps = compare_orthomosaic_to_basemap(
        str(ortho2_with_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset2_with_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho2_with_gcps_path}")
    metrics2_with_gcps = {}

print("\n✓ All comparisons complete!")


## Step 11: Generate Quality Report


In [ ]:
# Create comparison summary
import pandas as pd

# Helper function to safely get metrics
def get_metric(metrics_dict, key, default=0.0):
    """Safely get a metric value, handling different key formats."""
    if not metrics_dict:
        return default
    # Try direct key
    if key in metrics_dict:
        return metrics_dict[key]
    # Try with _pixels suffix
    if f"{key}_pixels" in metrics_dict:
        return metrics_dict[f"{key}_pixels"]
    return default

# Collect metrics
summary_data = {
    'Dataset': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2'],
    'GCPs Used': ['No', 'Yes', 'No', 'Yes'],
    'RMSE': [
        get_metric(metrics1_no_gcps, 'rmse'),
        get_metric(metrics1_with_gcps, 'rmse'),
        get_metric(metrics2_no_gcps, 'rmse'),
        get_metric(metrics2_with_gcps, 'rmse'),
    ],
    'MAE': [
        get_metric(metrics1_no_gcps, 'mae'),
        get_metric(metrics1_with_gcps, 'mae'),
        get_metric(metrics2_no_gcps, 'mae'),
        get_metric(metrics2_with_gcps, 'mae'),
    ],
    'Correlation': [
        get_metric(metrics1_no_gcps, 'correlation'),
        get_metric(metrics1_with_gcps, 'correlation'),
        get_metric(metrics2_no_gcps, 'correlation'),
        get_metric(metrics2_with_gcps, 'correlation'),
    ],
    'SSIM': [
        get_metric(metrics1_no_gcps, 'ssim'),
        get_metric(metrics1_with_gcps, 'ssim'),
        get_metric(metrics2_no_gcps, 'ssim'),
        get_metric(metrics2_with_gcps, 'ssim'),
    ],
    'Displacement (pixels)': [
        get_metric(metrics1_no_gcps, 'displacement_magnitude'),
        get_metric(metrics1_with_gcps, 'displacement_magnitude'),
        get_metric(metrics2_no_gcps, 'displacement_magnitude'),
        get_metric(metrics2_with_gcps, 'displacement_magnitude'),
    ],
    'Num Matches': [
        get_metric(metrics1_no_gcps, 'num_matches', default=0),
        get_metric(metrics1_with_gcps, 'num_matches', default=0),
        get_metric(metrics2_no_gcps, 'num_matches', default=0),
        get_metric(metrics2_with_gcps, 'num_matches', default=0),
    ],
}

df = pd.DataFrame(summary_data)

print("\n" + "=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
print(df.to_string(index=False))

# Save summary to CSV
summary_csv = output_dir / "quality_summary.csv"
df.to_csv(summary_csv, index=False)
print(f"\n✓ Summary saved to: {summary_csv}")

# Create visualizations
available_metrics = []
for col in ['RMSE', 'MAE', 'Correlation', 'SSIM', 'Displacement (pixels)', 'Num Matches']:
    if col in df.columns and df[col].sum() > 0:
        available_metrics.append(col)

if available_metrics:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, metric in enumerate(available_metrics[:6]):
        ax = axes[i]
        df_pivot = df.pivot(index='Dataset', columns='GCPs Used', values=metric)
        df_pivot.plot(kind='bar', ax=ax, rot=0)
        ax.set_title(metric)
        ax.set_ylabel('Value')
        ax.legend(title='GCPs Used')
        ax.grid(True, alpha=0.3)
    
    # Hide unused subplots
    for i in range(len(available_metrics), 6):
        axes[i].axis('off')
    
    plt.tight_layout()
    report_plot_path = output_dir / "quality_report.png"
    plt.savefig(report_plot_path, dpi=150, bbox_inches='tight')
    print(f"✓ Quality report plot saved to: {report_plot_path}")
    plt.show()
else:
    print("⚠️  No metrics available for visualization")

# Generate markdown report
report_md = output_dir / "quality_report.md"
with open(report_md, 'w') as f:
    f.write("# Westminster Ground Truth Analysis - Quality Report\n\n")
    f.write("## Summary\n\n")
    f.write(df.to_markdown(index=False))
    f.write("\n\n## Findings\n\n")
    f.write("### Dataset 1\n")
    f.write(f"- **Without GCPs**: RMSE={get_metric(metrics1_no_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics1_no_gcps, 'displacement_magnitude'):.2f} pixels\n")
    f.write(f"- **With GCPs**: RMSE={get_metric(metrics1_with_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics1_with_gcps, 'displacement_magnitude'):.2f} pixels\n\n")
    f.write("### Dataset 2\n")
    f.write(f"- **Without GCPs**: RMSE={get_metric(metrics2_no_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics2_no_gcps, 'displacement_magnitude'):.2f} pixels\n")
    f.write(f"- **With GCPs**: RMSE={get_metric(metrics2_with_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics2_with_gcps, 'displacement_magnitude'):.2f} pixels\n")

print(f"✓ Markdown report saved to: {report_md}")
